In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when
from pyspark.sql.functions import lit

In [14]:
business = spark.read.json("abfss://files@datalakehay9qx2.dfs.core.windows.net/data/yelp_academic_dataset_business.json")
display(business.limit(10))

In [15]:
business_df = business_df.drop("hours").drop("attributes")

business_df = business_df.filter(col("categories").isNotNull())
#business_df = business_df.withColumn("name", F.regexp_replace("name", '"', ""))
#business_df = business_df.withColumn("address", F.regexp_replace("address", '"', "")),

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA",
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

# Filter for businesses in US states
usa = business_df.filter(col("state").isin(states))

display(usa.limit(10))

In [16]:
us_restaurants = usa.filter(usa['categories'].contains('Restaurants'))
us_restaurants = us_restaurants.withColumn("category", lit(None))

us_restaurants = us_restaurants.withColumn("category", 
                                            when(col("categories").contains("Korean"), "Korean")\
 
                                            .when(col("categories").contains("Japanese"), "Japanese")\
 
                                            .when(col("categories").contains("Chinese"), "Chinese")\

                                            .when(col("categories").contains("Vietnamese"), "Vietnamese")\

                                            .when(col("categories").contains("Thai"), "Thai")\
  
                                            .when(col("categories").contains("Italian"), "Italian")\

                                            .when(col("categories").contains("French"), "French")
                                            )                                                                                                                                                                                                               

display(us_restaurants.limit(10))        

In [17]:
us_restaurants = us_restaurants.drop("categories")
us_restaurants = us_restaurants.filter(col("category").isNotNull())

display(us_restaurants.limit(10))   

In [18]:
## check total number of us restaurants
us_restaurants.count()

In [43]:
## check whether has duplicated business id
us_restaurants.groupBy("business_id").count().filter(col("count") > 1).count()

In [38]:
us_restaurants.write.save(path="csv", format='csv', mode='overwrite', sep='\t')

In [32]:
us_restaurants.write.option("database", "YelpDB").mode("overwrite").saveAsTable("us_restaurants")